In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [3]:
from pyspark import SparkContext
sc = spark.sparkContext

In [4]:
raw_data = sc.textFile("Lottery.txt")

In [5]:
raw_data.flatMap(lambda x: x.split('NumaraSirasi')).filter(lambda x: "devretti" in x).take(2000)[500]

'": "5 - 7 - 36 - 39 - 47 - 48", "devretti": true, "buyukIkrKazananIlIlceler": [], "hafta": 501, "buyukIkramiyeKazananIl": ""}, "success": true}{"data": {"bilenKisiler": [{"kisiBasinaDusenIkramiye": 1341977.1, "tur": "$6_BILEN", "oid": "sl_2006-06-24_6", "kisiSayisi": 2}, {"kisiBasinaDusenIkramiye": 1429.85, "tur": "$5_BILEN", "oid": "sl_2006-06-24_5", "kisiSayisi": 590}, {"kisiBasinaDusenIkramiye": 13.55, "tur": "$4_BILEN", "oid": "sl_2006-06-24_4", "kisiSayisi": 29130}, {"kisiBasinaDusenIkramiye": 2.2, "tur": "$3_BILEN", "oid": "sl_2006-06-24_3", "kisiSayisi": 481529}], "oid": "sl_2006-06-24", "devirSayisi": 0, "cekilisTuru": "SAYISAL_LOTO", "rakamlar": "15#24#25#31#32#41", "cekilisTarihi": "24/06/2006", "rakamlar'

In [6]:
raw_data.flatMap(lambda x: x.split("rakamlarNumaraSirasi")) \
        .filter(lambda x: "devretti" in x) \
        .map(lambda x: x.split(",")) \
        .map(lambda x: x[0]) \
        .map(lambda x: x.replace('": "',"")) \
        .map(lambda x: x.replace('"',"")) \
        .map(lambda x: x.split(" - ")).take(10)

[['5', '15', '20', '32', '34', '47'],
 ['3', '14', '16', '23', '27', '43'],
 ['6', '22', '26', '31', '38', '44'],
 ['7', '11', '16', '18', '25', '26'],
 ['2', '5', '14', '30', '31', '39'],
 ['2', '14', '24', '25', '30', '36'],
 ['4', '8', '14', '24', '27', '36'],
 ['3', '17', '36', '41', '46', '47'],
 ['4', '15', '16', '18', '27', '41'],
 ['5', '11', '24', '29', '41', '45']]

In [7]:
take_numbers = raw_data.flatMap(lambda x: x.split("rakamlarNumaraSirasi")) \
                       .filter(lambda x: "devretti" in x) \
                       .map(lambda x: x.split(",")) \
                       .map(lambda x: x[0]) \
                       .map(lambda x: x.replace('": "',"")) \
                       .map(lambda x: x.replace('"',"")) \
                       .map(lambda x: x.split(" - ")).collect()

In [8]:
take_numbers[0]

['5', '15', '20', '32', '34', '47']

In [9]:
list_value = [[int(i) for i in list_] for list_ in take_numbers]
list_value[0]

[5, 15, 20, 32, 34, 47]

In [10]:
list_value = sc.parallelize(list_value)
list_value.take(10)

[[5, 15, 20, 32, 34, 47],
 [3, 14, 16, 23, 27, 43],
 [6, 22, 26, 31, 38, 44],
 [7, 11, 16, 18, 25, 26],
 [2, 5, 14, 30, 31, 39],
 [2, 14, 24, 25, 30, 36],
 [4, 8, 14, 24, 27, 36],
 [3, 17, 36, 41, 46, 47],
 [4, 15, 16, 18, 27, 41],
 [5, 11, 24, 29, 41, 45]]

In [11]:
from itertools import combinations

def comb(rdd):
  triple = list(combinations(rdd, 3))  
  return triple

tripleRDD = list_value.flatMap(lambda x: comb(x))
tripleRDD.take(30)

[(5, 15, 20),
 (5, 15, 32),
 (5, 15, 34),
 (5, 15, 47),
 (5, 20, 32),
 (5, 20, 34),
 (5, 20, 47),
 (5, 32, 34),
 (5, 32, 47),
 (5, 34, 47),
 (15, 20, 32),
 (15, 20, 34),
 (15, 20, 47),
 (15, 32, 34),
 (15, 32, 47),
 (15, 34, 47),
 (20, 32, 34),
 (20, 32, 47),
 (20, 34, 47),
 (32, 34, 47),
 (3, 14, 16),
 (3, 14, 23),
 (3, 14, 27),
 (3, 14, 43),
 (3, 16, 23),
 (3, 16, 27),
 (3, 16, 43),
 (3, 23, 27),
 (3, 23, 43),
 (3, 27, 43)]

In [12]:
most_triple = tripleRDD.map(lambda x: (x,1))
most_triple.take(30)

[((5, 15, 20), 1),
 ((5, 15, 32), 1),
 ((5, 15, 34), 1),
 ((5, 15, 47), 1),
 ((5, 20, 32), 1),
 ((5, 20, 34), 1),
 ((5, 20, 47), 1),
 ((5, 32, 34), 1),
 ((5, 32, 47), 1),
 ((5, 34, 47), 1),
 ((15, 20, 32), 1),
 ((15, 20, 34), 1),
 ((15, 20, 47), 1),
 ((15, 32, 34), 1),
 ((15, 32, 47), 1),
 ((15, 34, 47), 1),
 ((20, 32, 34), 1),
 ((20, 32, 47), 1),
 ((20, 34, 47), 1),
 ((32, 34, 47), 1),
 ((3, 14, 16), 1),
 ((3, 14, 23), 1),
 ((3, 14, 27), 1),
 ((3, 14, 43), 1),
 ((3, 16, 23), 1),
 ((3, 16, 27), 1),
 ((3, 16, 43), 1),
 ((3, 23, 27), 1),
 ((3, 23, 43), 1),
 ((3, 27, 43), 1)]

In [13]:
most_triple = most_triple.reduceByKey(lambda x,y: x+y)
most_triple = most_triple.sortBy(lambda x: x[1], ascending=False)
most_triple.take(100)

[((3, 8, 40), 8),
 ((12, 19, 47), 8),
 ((1, 21, 24), 8),
 ((23, 39, 47), 7),
 ((10, 38, 41), 7),
 ((1, 24, 42), 7),
 ((4, 9, 22), 7),
 ((18, 32, 44), 7),
 ((5, 38, 41), 7),
 ((7, 17, 20), 7),
 ((18, 28, 36), 7),
 ((36, 38, 41), 6),
 ((4, 39, 42), 6),
 ((1, 39, 47), 6),
 ((9, 16, 46), 6),
 ((6, 20, 47), 6),
 ((16, 17, 48), 6),
 ((11, 18, 32), 6),
 ((1, 16, 44), 6),
 ((1, 18, 42), 6),
 ((9, 34, 36), 6),
 ((11, 16, 44), 6),
 ((7, 10, 20), 6),
 ((20, 38, 47), 6),
 ((11, 18, 41), 6),
 ((11, 30, 41), 6),
 ((22, 25, 27), 6),
 ((7, 16, 17), 6),
 ((14, 20, 38), 6),
 ((2, 14, 38), 6),
 ((6, 16, 44), 6),
 ((11, 40, 45), 6),
 ((17, 21, 26), 6),
 ((40, 41, 45), 6),
 ((30, 38, 40), 6),
 ((5, 47, 48), 6),
 ((12, 34, 46), 6),
 ((10, 16, 48), 6),
 ((10, 36, 40), 6),
 ((2, 22, 38), 6),
 ((23, 26, 39), 6),
 ((13, 32, 49), 6),
 ((16, 36, 48), 6),
 ((14, 34, 40), 6),
 ((22, 23, 35), 6),
 ((2, 14, 31), 5),
 ((2, 30, 39), 5),
 ((14, 30, 31), 5),
 ((4, 14, 27), 5),
 ((5, 41, 45), 5),
 ((18, 40, 41), 5),
 ((18

**(3, 8, 40) , (12, 19, 47) and 1, 21, 24) 	top most drawn	triple numbers that occured 8 times**
